----------------------------------------------------------------------------------Staging Layer----------------------------------------------------------------------------------

In [11]:
import pandas as pd
employee = pd.read_csv('../Source/employee.csv')
course = pd.read_csv('../Source/courses.csv')
employee_course = pd.read_csv('../Source/employee_course.csv')

In [12]:
employee_df = pd.DataFrame(employee)
course_df = pd.DataFrame(course)
employee_course_df = pd.DataFrame(employee_course)

In [13]:
print(employee.info())
print(course.info())
print(employee_course_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                500 non-null    int64 
 1   empID             500 non-null    object
 2   name              500 non-null    object
 3   designation       500 non-null    object
 4   password          500 non-null    object
 5   email             500 non-null    object
 6   mobile            500 non-null    int64 
 7   performance_rate  500 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 31.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                50 non-null     int64 
 1   courseID          50 non-null     object
 2   name              50 non-null     object
 3   duration          50 non-null     object
 4

In [15]:
employee_df.to_csv('Staging_Layer/employee.csv', index=False)
course_df.to_csv('Staging_Layer/course.csv', index=False)
employee_course_df.to_csv('Staging_Layer/employee_course.csv',index=False)

---------------------------------------------------------------------------------Mart Layer---------------------------------------------------------------------------------

Course Table

In [44]:
course_mart_df = pd.read_csv('Staging_Layer/course.csv')
course_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                50 non-null     int64 
 1   courseID          50 non-null     object
 2   name              50 non-null     object
 3   duration          50 non-null     object
 4   difficulty_level  50 non-null     object
 5   learning_path     50 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.5+ KB


In [41]:
course_mart_df.dropna(subset=['courseID'], inplace=True) # Drop rows with null courseID
course_mart_df['name'].fillna('Unknown', inplace=True)  # Filling with 'Unknown' for name
course_mart_df['duration'].fillna('Not Specified', inplace=True)  # Fill with 'Not Specified'
course_mart_df['difficulty_level'].fillna('Not Specified', inplace=True)  # Fill with 'Not Specified'
course_mart_df['learning_path'].fillna('Not Assigned', inplace=True)  # Fill with 'Not Assigned'
course_mart_df=course_mart_df.drop_duplicates() # Dropping duplicate values

C:\Users\AkhilKumar\AppData\Local\Temp\ipykernel_2588\2640944512.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  course_mart_df['courseID'].fillna(0, inplace=True)  # Filling with 0 for courseID
C:\Users\AkhilKumar\AppData\Local\Temp\ipykernel_2588\2640944512.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

In [42]:
def convert_duration_to_weeks(duration):
    if isinstance(duration, str):
        # Extract number and convert to weeks based on context
        if 'week' in duration.lower():
            return int(duration.split()[0])  # Returns the numeric part
        elif 'month' in duration.lower():
            return int(duration.split()[0]) * 4  # Assuming 1 month = 4 weeks
        elif 'year' in duration.lower():
            return int(duration.split()[0]) * 52 # Assuming 1 year = 52 weeks
    return None  # For 'Not Specified' or other values

# Apply conversion to a new column for numerical representation
course_mart_df['duration_weeks'] = course_mart_df['duration'].apply(convert_duration_to_weeks)

In [43]:
course_mart_df.to_csv('Mart_Layer/course.csv',index=False)

Employee_Course Table

In [45]:
employee_course_mart_df=pd.read_csv('Staging_Layer/employee_course.csv')
employee_course_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   empID            2000 non-null   object 
 1   courseID         2000 non-null   object 
 2   completion_rate  2000 non-null   float64
dtypes: float64(1), object(2)
memory usage: 47.0+ KB


In [ ]:
employee_course_mart_df=employee_course_mart_df.drop_duplicates() # Dropping duplicate values

In [54]:
# Moving data into Mart Layer
employee_course_mart_df.to_csv('Mart_Layer/employee_course.csv', index=False)

Employee Table

In [83]:
employee_mart_df = pd.read_csv('Staging_Layer/employee.csv')
employee_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                500 non-null    int64 
 1   empID             500 non-null    object
 2   name              500 non-null    object
 3   designation       500 non-null    object
 4   password          500 non-null    object
 5   email             500 non-null    object
 6   mobile            500 non-null    int64 
 7   performance_rate  500 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 31.4+ KB


In [84]:
# Dropping performance_rate column 
employee_mart_df.drop('performance_rate', axis=1, inplace=True)

# Calculating mean of completion_rates for each employee
mean_completion_rate = employee_course_mart_df.groupby('empID')['completion_rate'].mean().reset_index()

# Rename the completion_rate column to performance_rate for clarity
mean_completion_rate.rename(columns={'completion_rate': 'performance_rate'}, inplace=True)

# Step 2: Merge the mean completion rate back to the employee mart DataFrame
merged_df = employee_mart_df.merge(mean_completion_rate, on='empID', how='left')

# Assign the performance_rate from the merged DataFrame back to employee_mart_df
employee_mart_df['performance_rate'] = merged_df['performance_rate']

employee_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    int64  
 1   empID             500 non-null    object 
 2   name              500 non-null    object 
 3   designation       500 non-null    object 
 4   password          500 non-null    object 
 5   email             500 non-null    object 
 6   mobile            500 non-null    int64  
 7   performance_rate  494 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 31.4+ KB


In [85]:
employee_mart_df.dropna(subset=['empID'], inplace=True)
employee_mart_df['name'].fillna('Unknown',inplace=True)
employee_mart_df['designation'].fillna('Not Known', inplace=True)
employee_mart_df['email'].fillna("Not Known", inplace=True)
employee_mart_df['password'].fillna(employee_mart_df['empID'],inplace=True)
employee_mart_df['mobile'].fillna(0,inplace=True)
employee_mart_df['performance_rate'].fillna(0,inplace=True)
employee_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    int64  
 1   empID             500 non-null    object 
 2   name              500 non-null    object 
 3   designation       500 non-null    object 
 4   password          500 non-null    object 
 5   email             500 non-null    object 
 6   mobile            500 non-null    int64  
 7   performance_rate  500 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 31.4+ KB


C:\Users\AkhilKumar\AppData\Local\Temp\ipykernel_2588\667963262.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  employee_mart_df['name'].fillna('Unknown',inplace=True)
C:\Users\AkhilKumar\AppData\Local\Temp\ipykernel_2588\667963262.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [87]:
# Dropping duplicates
employee_mart_df=employee_mart_df.drop_duplicates()

# Rounding performance_rate to two decimals
employee_mart_df['performance_rate'] = employee_mart_df['performance_rate'].round(2)

In [88]:
# Moving data into mart layer
employee_mart_df.to_csv('Mart_Layer/employee.csv')